In [3]:
import sys,os,datetime,shutil
import pandas as pd
from pathlib import Path

project_root = os.path.abspath(Path("../."))
sys.path.append(str(project_root))
print(project_root)
from scripts import s4_baselines_eval,s5_EAF_eval
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

##################################
load_mt5_data=False
Long_allowed=True
Short_allowed=False
list_target_isClose=[True,False] 

extra_symbols= []
base_features=['Close', 'High', 'Log_Return', 'Log_Return_change1', 'Log_Return_change2']
tsfresh_features = []
##################################

cur_date = datetime.date.today().strftime('%Y%m%d')
results_folder = Path(project_root) / f'results/performance2/baselines{cur_date}'

# if os.path.exists(results_folder):
#     shutil.rmtree(results_folder) ## remove last results and create for new run

results_folder.mkdir(parents=True, exist_ok=True)

for target_isClose in list_target_isClose:
    features_file = f"gold_price_features_{'Close' if target_isClose else 'LogReturn'}{'_mt5' if load_mt5_data else ''}.csv"
    features_filepath=str(project_root) + f'/data/features/{features_file}'
    df_OHLCV =pd.read_csv(
        features_filepath,
        index_col=0, parse_dates=True)
    
    df_OHLCV.index.rename('Date', inplace=True)
    if 'Date' in df_OHLCV.columns:
        df_OHLCV['Date'] = pd.to_datetime(df_OHLCV['Date'])
        df_OHLCV.set_index('Date', inplace=True)
    ### Run ####

    list_results=[]
    for GLOBAL_SEED in [10,200,3000,40000,500000]: #
        print("target_isClose = ", target_isClose, " | ", "GLOBAL_SEED = ",GLOBAL_SEED)

        baseline5_BuyAndHold= s4_baselines_eval.run5_BuyAndHold(GLOBAL_SEED,target_isClose,features_filepath,Long_allowed,Short_allowed)
        print(baseline5_BuyAndHold)
        with open(results_folder / f'baseline5_BuyAndHold{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
            f.writelines(str(baseline5_BuyAndHold))

        baseline2_ARIMA= s4_baselines_eval.run2_ARIMA(GLOBAL_SEED,target_isClose,features_filepath,Long_allowed,Short_allowed)
        print(baseline2_ARIMA)
        with open(results_folder / f'baseline2_ARIMA{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
            f.writelines(str(baseline2_ARIMA))

        baseline3_LSTM= s4_baselines_eval.run3_LSTM_scaled(GLOBAL_SEED,target_isClose,features_filepath,Long_allowed,Short_allowed,write_OHLCV_output=False)
        print(baseline3_LSTM)
    
        with open(results_folder / f'baseline3_LSTM{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
            f.writelines(str(baseline3_LSTM))
            
        baseline4_LGBM= s4_baselines_eval.run4_LGBM(GLOBAL_SEED,target_isClose,features_filepath,Long_allowed,Short_allowed)
        print(baseline4_LGBM)
        with open(results_folder / f'baseline4_LGBM{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
            f.writelines(str(baseline4_LGBM))

        optuna_search=False ### to reduce running time, just load existing hyperparameters - already optimized
        baseline1_XGBM= s4_baselines_eval.run1_XGBM(GLOBAL_SEED,optuna_search,target_isClose,features_filepath,Long_allowed,Short_allowed)
        print(baseline1_XGBM)
        with open(results_folder / f'baseline1_XGBM{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
            f.writelines(str(baseline1_XGBM))

        list_adwin_config = ['adwin_default'] # ['adwin_Config_B','adwin_Config_C','adwin_Config_D']
        for adwin_config  in list_adwin_config:
            EAF_metrics,final_model,drift_dates_test,X_test_eaf, y_test_eaf= s5_EAF_eval.run_adpative(GLOBAL_SEED,target_isClose,features_filepath,Long_allowed,Short_allowed,adwin_config)
            print(EAF_metrics)
            list_results.append(EAF_metrics)
            with open(results_folder / f'EAF_metrics_{adwin_config}_{"Close" if target_isClose else "LogReturn"}.txt','a') as f:
                f.writelines(str(EAF_metrics))
            


g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis
target_isClose =  True  |  GLOBAL_SEED =  10
run5_BuyAndHold ...
Loading feature data from g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis/data/features/gold_price_features_Close.csv
Preparing time series data with historical context
[{'GLOBAL_SEED': 10, 'baseline_modelName': 'baseline_BuyAndHold', 'target_isClose': True, 'period': 'training', 'rmse': 780.616931906994, 'mae': 716.2540588940235, 'r2': -5.324936074250366, 'strategy_name': 'baseline_BuyAndHold', 'strategy_type': 'moc', 'n_days': 3262, 'trading_days': 3261, 'skipped_trades': 0, 'gap_skips': 0, 'edge_skips': 0, 'trade_rate': 0.9996934396076027, 'win_days': 1679, 'win_rate': 0.5148727384237963, 'compound_gross_return': 1.331388067276047, 'total_return': 1.0769871552611778, 'annualized_return': 0.06757749178506645, 'cagr': 0.06744424939914984, 'sharpe_ratio': 0.441307361189705, 'sortino_ratio': 0.43130743100